In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from urllib.parse import unquote_to_bytes

2024-10-17 13:35:06.009922: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 13:35:06.013188: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 13:35:06.023662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 13:35:06.041086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 13:35:06.046146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 13:35:06.058806: I tensorflow/core/platform/cpu_feature_gu

In [2]:
# Example data

data = [
    { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2D', 'attack':1 }
]

In [3]:
# Prepare Data 
maxlen = 1024 
X = [] 
Y = [] 

def decode_query(str): 
    return unquote_to_bytes(str.replace('+',' ')) 

for item in data: 
    arr = decode_query(item['str'])[:maxlen] 
    arrlen = len(arr) 
    seq = [0] * maxlen 
    for i in range(arrlen): 
        seq[maxlen - arrlen + i] = arr[i] 
    X.append(seq) 
    Y.append(item['attack'])

In [8]:
# 
# Build Model (Simple LSTM) 
# 

model = tf.keras.Sequential([ 
    layers.Input(shape=(maxlen,), batch_size=1), 
    layers.Embedding(256, 32), 
    layers.LSTM(16), 
    layers.Dense(1, activation='sigmoid')]) 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 1024, 32)          │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 16)                │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 1)                 │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,345 (44.32 KB)

 Trainable params: 11,345 (44.32 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#      
# Train Model 
# 

model.fit(np.asarray(X).astype(np.float32), 
          np.asarray(Y).astype(np.float32), 
          epochs=100, batch_size=1)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 1.0000 - loss: 0.6825
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 1.0000 - loss: 0.6764
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 1.0000 - loss: 0.6698
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 1.0000 - loss: 0.6617
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 1.0000 - loss: 0.6539
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 1.0000 - loss: 0.6458
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.6441
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 1.0000 - loss: 0.6361
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 1.0000 - loss: 0.6196
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 1.0000 - loss: 0.6084
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 1.0000 - loss: 0.6092
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 1

In [12]:
# 
# Save Model 
# 

model.export('model') 

converter = tf.lite.TFLiteConverter.from_saved_model('model')

converter.experimental_enable_resource_variables = True

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

snort_model = converter.convert() 

with open('snort.model', 'wb') as f: 
    f.write(snort_model)

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


Saved artifact at 'model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  123857746066800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123857746065040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123857746059232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123857746064160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123857746066624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123857746399424: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1729165621.261225   26662 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1729165621.261239   26662 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-17 13:47:01.261401: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: model
2024-10-17 13:47:01.262147: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-17 13:47:01.262158: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: model
2024-10-17 13:47:01.267691: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-17 13:47:01.289554: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: model
2024-10-17 13:47:01.298700: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 37302 microseconds.
2024-10-17 13:47:01.347540: W tensorflow/compiler/mlir/lite/flatbuffer_export.